In [34]:
from ioos_qc import qartod
import pandas as pd
import numpy as np

from sklearn import preprocessing
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras import regularizers
from tensorflow.keras.layers.experimental.preprocessing import Normalization
from tensorflow.keras import models
from tensorflow.keras import layers

In [22]:
argo_df = pd.read_csv('data_for_spikes.csv')

argo_df = argo_df.drop(argo_df.columns[0],axis=1)

In [23]:
argo_df

,10.0,11.0,12.0,13.0,14.0,15.0,16.0,17.0,18.0,19.0,...,92.0,93.0,94.0,95.0,96.0,97.0,98.0,99.0,100.0,101.0
0,26.594000,26.593000,26.592000,26.591000,26.590000,26.586500,26.583000,26.582000,26.5810,26.5800,...,21.6040,21.6865,21.769000,21.669500,21.5700,21.3615,21.153,21.1450,21.137,21.137
1,24.633000,24.633500,24.634000,24.633500,24.633000,24.633500,24.634000,24.633000,24.6320,24.6310,...,24.4360,24.4360,24.436000,24.437500,24.4390,24.4405,24.442,24.4415,24.441,24.441
2,26.387000,26.386500,26.386000,26.385500,26.385000,26.385000,26.385000,26.385000,26.3850,26.3815,...,26.0130,25.7725,25.532000,25.352000,25.1720,24.9935,24.815,24.3585,23.902,23.902
3,21.368000,21.359500,21.351000,21.321000,21.291000,21.214000,21.137000,20.997500,20.8580,20.7185,...,18.9910,18.9765,18.962000,18.958000,18.9540,18.9365,18.919,18.9190,18.919,18.919
4,26.536000,26.533833,26.531667,26.529500,26.527333,26.525167,26.523000,26.516800,26.5106,26.5044,...,24.6395,24.5470,24.454500,24.362000,24.3620,24.3620,24.362,24.3620,24.362,24.362
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7125,9.654000,9.653500,9.653000,9.652000,9.654000,9.656000,9.656333,9.656667,9.6570,9.6570,...,9.3490,9.2750,9.229000,9.183000,9.1750,9.1670,9.159,9.1630,9.163,9.163
7126,9.485000,9.485000,9.486000,9.487000,9.486500,9.486000,9.486000,9.486000,9.4860,9.4865,...,9.0435,9.0160,9.008000,9.000000,8.9865,8.9730,8.968,8.9630,8.958,8.958
7127,8.909667,8.907333,8.905000,8.903500,8.902000,8.902000,8.902000,8.905500,8.9090,8.9120,...,9.0910,9.0620,9.059667,9.057333,9.0550,9.0010,8.947,8.9140,8.914,8.914
7128,9.567000,9.561000,9.558667,9.556333,9.554000,9.513500,9.473000,9.447000,9.3935,9.3400,...,8.9810,8.9670,8.971500,8.976000,8.9840,8.9920,8.993,8.9940,8.994,8.994


In [24]:
noise = np.random.normal(0,2,[len(argo_df.index),len(argo_df.columns)]) # creating a dataframe of normally distributed values around 0, with a stndev of 2. Stndev can be changed to make more or less noisy data

argo_noise = argo_df+noise

argo_noise.to_csv('argo_noise.csv',index=False)

spike_flags = pd.DataFrame(index=argo_noise.index,columns=argo_noise.columns)
range_flags = pd.DataFrame(index=argo_noise.index,columns=argo_noise.columns)

In [25]:
for index in argo_noise.index:
   flags = qartod.spike_test(inp=argo_noise.iloc[index],suspect_threshold=3.0,fail_threshold=8.0) # using 3°C as suspect and 8°C as fail, per IOOS recommendations
   spike_flags.iloc[index] = flags
   flags = qartod.gross_range_test(inp=argo_noise.iloc[index],fail_span=[0,30]) #flags any value temp that is less than 0°C or higher than 30°C
   range_flags.iloc[index] = flags 

spike_flags.to_csv('spike_flags.csv',index=False)
range_flags.to_csv('range_flags.csv',index=False)

In [26]:
spike_flags[range_flags == 4] = 4

In [65]:
spike_flags[spike_flags == 2] = 1
spike_flags[spike_flags == 3] = 1
spike_flags[spike_flags == 4] = 0
spike_flags

,10.0,11.0,12.0,13.0,14.0,15.0,16.0,17.0,18.0,19.0,...,92.0,93.0,94.0,95.0,96.0,97.0,98.0,99.0,100.0,101.0
0,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
1,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
2,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
3,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
4,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7125,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
7126,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
7127,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
7128,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,0,1


In [66]:
y = spike_flags.values.astype('float')

In [67]:
Xscaled = preprocessing.scale(argo_df.values)

In [68]:
Xscaled.shape

(7130, 92)

In [69]:
def initialize_model():
    model = models.Sequential()

    model.add(layers.Dense(10, activation='relu', input_dim=92)) # First hidden layer with 5 neurons
    model.add(layers.Dense(20, activation='relu')) # First hidden layer with 5 neurons
    model.add(layers.Dense(7, activation='relu')) # First hidden layer with 5 neurons
    model.add(layers.Dense(92, activation='sigmoid'))
    
    return model

In [70]:
def compile_model(model):
    model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

    return model

In [71]:
es = EarlyStopping(patience=30, restore_best_weights=True)

model = initialize_model()

model = compile_model(model)

In [72]:
history = model.fit(Xscaled, y,
                    validation_split=0.3,
                    epochs=500,
                    batch_size=16, 
                    verbose=1)

Epoch 1/500
312/312 [==============================] - 2s 4ms/step - loss: 0.2384 - accuracy: 0.0000e+00 - val_loss: 0.0667 - val_accuracy: 0.0000e+00
Epoch 2/500
312/312 [==============================] - 1s 3ms/step - loss: 0.0980 - accuracy: 0.0000e+00 - val_loss: 0.0277 - val_accuracy: 0.0000e+00
Epoch 3/500
312/312 [==============================] - 1s 3ms/step - loss: 0.0746 - accuracy: 0.0000e+00 - val_loss: 0.0196 - val_accuracy: 0.0000e+00
Epoch 4/500
312/312 [==============================] - 1s 3ms/step - loss: 0.0665 - accuracy: 0.0000e+00 - val_loss: 0.0170 - val_accuracy: 0.0000e+00
Epoch 5/500
312/312 [==============================] - 1s 3ms/step - loss: 0.0622 - accuracy: 0.0000e+00 - val_loss: 0.0195 - val_accuracy: 0.0000e+00
Epoch 6/500
312/312 [==============================] - 1s 3ms/step - loss: 0.0598 - accuracy: 0.0000e+00 - val_loss: 0.0156 - val_accuracy: 0.0000e+00
Epoch 7/500
312/312 [==============================] - 1s 3ms/step - loss: 0.0573 - accuracy: 

312/312 [==============================] - 1s 3ms/step - loss: 0.0488 - accuracy: 2.0036e-04 - val_loss: 0.0115 - val_accuracy: 0.0000e+00
Epoch 57/500
312/312 [==============================] - 1s 4ms/step - loss: 0.0488 - accuracy: 0.0000e+00 - val_loss: 0.0116 - val_accuracy: 0.0000e+00
Epoch 58/500
312/312 [==============================] - 1s 3ms/step - loss: 0.0488 - accuracy: 0.0000e+00 - val_loss: 0.0117 - val_accuracy: 0.0000e+00
Epoch 59/500
312/312 [==============================] - 1s 3ms/step - loss: 0.0488 - accuracy: 0.0000e+00 - val_loss: 0.0114 - val_accuracy: 0.0000e+00
Epoch 60/500
312/312 [==============================] - 1s 3ms/step - loss: 0.0487 - accuracy: 0.0000e+00 - val_loss: 0.0114 - val_accuracy: 0.0000e+00
Epoch 61/500
312/312 [==============================] - 1s 3ms/step - loss: 0.0488 - accuracy: 0.0000e+00 - val_loss: 0.0115 - val_accuracy: 0.0000e+00
Epoch 62/500
312/312 [==============================] - 1s 4ms/step - loss: 0.0487 - accuracy: 0.0000

Epoch 110/500
312/312 [==============================] - 1s 3ms/step - loss: 0.0481 - accuracy: 0.0000e+00 - val_loss: 0.0113 - val_accuracy: 0.0000e+00
Epoch 111/500
312/312 [==============================] - 1s 3ms/step - loss: 0.0479 - accuracy: 0.0000e+00 - val_loss: 0.0112 - val_accuracy: 0.0000e+00
Epoch 112/500
312/312 [==============================] - 1s 3ms/step - loss: 0.0482 - accuracy: 2.0036e-04 - val_loss: 0.0114 - val_accuracy: 0.0000e+00
Epoch 113/500
312/312 [==============================] - 1s 3ms/step - loss: 0.0480 - accuracy: 0.0000e+00 - val_loss: 0.0113 - val_accuracy: 0.0000e+00
Epoch 114/500
312/312 [==============================] - 1s 3ms/step - loss: 0.0479 - accuracy: 0.0000e+00 - val_loss: 0.0113 - val_accuracy: 0.0000e+00
Epoch 115/500
312/312 [==============================] - 1s 3ms/step - loss: 0.0479 - accuracy: 2.0036e-04 - val_loss: 0.0113 - val_accuracy: 0.0000e+00
Epoch 116/500
312/312 [==============================] - 1s 4ms/step - loss: 0.047

312/312 [==============================] - 1s 3ms/step - loss: 0.0478 - accuracy: 2.0036e-04 - val_loss: 0.0114 - val_accuracy: 0.0000e+00
Epoch 164/500
312/312 [==============================] - 1s 3ms/step - loss: 0.0477 - accuracy: 2.0036e-04 - val_loss: 0.0113 - val_accuracy: 0.0000e+00
Epoch 165/500
312/312 [==============================] - 1s 3ms/step - loss: 0.0478 - accuracy: 2.0036e-04 - val_loss: 0.0113 - val_accuracy: 0.0000e+00
Epoch 166/500
312/312 [==============================] - 1s 3ms/step - loss: 0.0476 - accuracy: 2.0036e-04 - val_loss: 0.0113 - val_accuracy: 0.0000e+00
Epoch 167/500
312/312 [==============================] - 1s 3ms/step - loss: 0.0478 - accuracy: 2.0036e-04 - val_loss: 0.0113 - val_accuracy: 0.0000e+00
Epoch 168/500
312/312 [==============================] - 1s 3ms/step - loss: 0.0475 - accuracy: 2.0036e-04 - val_loss: 0.0113 - val_accuracy: 0.0000e+00
Epoch 169/500
312/312 [==============================] - 1s 3ms/step - loss: 0.0475 - accuracy: 

Epoch 217/500
312/312 [==============================] - 1s 3ms/step - loss: 0.0475 - accuracy: 0.0016 - val_loss: 0.0113 - val_accuracy: 0.0000e+00
Epoch 218/500
312/312 [==============================] - 1s 3ms/step - loss: 0.0475 - accuracy: 6.0108e-04 - val_loss: 0.0113 - val_accuracy: 0.0000e+00
Epoch 219/500
312/312 [==============================] - 1s 3ms/step - loss: 0.0474 - accuracy: 0.0012 - val_loss: 0.0114 - val_accuracy: 0.0164
Epoch 220/500
312/312 [==============================] - 1s 3ms/step - loss: 0.0475 - accuracy: 0.0010 - val_loss: 0.0113 - val_accuracy: 0.0033
Epoch 221/500
312/312 [==============================] - 1s 3ms/step - loss: 0.0476 - accuracy: 6.0108e-04 - val_loss: 0.0114 - val_accuracy: 9.3502e-04
Epoch 222/500
312/312 [==============================] - 1s 3ms/step - loss: 0.0474 - accuracy: 6.0108e-04 - val_loss: 0.0113 - val_accuracy: 0.0028
Epoch 223/500
312/312 [==============================] - 1s 3ms/step - loss: 0.0475 - accuracy: 0.0062 - v

Epoch 272/500
312/312 [==============================] - 1s 3ms/step - loss: 0.0472 - accuracy: 0.0014 - val_loss: 0.0113 - val_accuracy: 0.0084
Epoch 273/500
312/312 [==============================] - 1s 3ms/step - loss: 0.0474 - accuracy: 4.0072e-04 - val_loss: 0.0113 - val_accuracy: 0.0000e+00
Epoch 274/500
312/312 [==============================] - 1s 3ms/step - loss: 0.0475 - accuracy: 0.0014 - val_loss: 0.0113 - val_accuracy: 0.0000e+00
Epoch 275/500
312/312 [==============================] - 1s 3ms/step - loss: 0.0474 - accuracy: 4.0072e-04 - val_loss: 0.0114 - val_accuracy: 0.0000e+00
Epoch 276/500
312/312 [==============================] - 1s 3ms/step - loss: 0.0472 - accuracy: 6.0108e-04 - val_loss: 0.0114 - val_accuracy: 0.0000e+00
Epoch 277/500
312/312 [==============================] - 1s 3ms/step - loss: 0.0473 - accuracy: 4.0072e-04 - val_loss: 0.0113 - val_accuracy: 0.0023
Epoch 278/500
312/312 [==============================] - 1s 3ms/step - loss: 0.0477 - accuracy: 0.

312/312 [==============================] - 1s 4ms/step - loss: 0.0472 - accuracy: 0.0012 - val_loss: 0.0113 - val_accuracy: 0.0084
Epoch 328/500
312/312 [==============================] - 1s 3ms/step - loss: 0.0472 - accuracy: 0.0020 - val_loss: 0.0113 - val_accuracy: 0.0014
Epoch 329/500
312/312 [==============================] - 1s 4ms/step - loss: 0.0473 - accuracy: 0.0024 - val_loss: 0.0113 - val_accuracy: 0.0154
Epoch 330/500
312/312 [==============================] - 1s 4ms/step - loss: 0.0472 - accuracy: 2.0036e-04 - val_loss: 0.0113 - val_accuracy: 0.0126
Epoch 331/500
312/312 [==============================] - 1s 3ms/step - loss: 0.0472 - accuracy: 0.0020 - val_loss: 0.0114 - val_accuracy: 0.0000e+00
Epoch 332/500
312/312 [==============================] - 1s 3ms/step - loss: 0.0471 - accuracy: 0.0070 - val_loss: 0.0114 - val_accuracy: 4.6751e-04
Epoch 333/500
312/312 [==============================] - 1s 3ms/step - loss: 0.0472 - accuracy: 0.0018 - val_loss: 0.0113 - val_accu

Epoch 383/500
312/312 [==============================] - 1s 3ms/step - loss: 0.0470 - accuracy: 0.0142 - val_loss: 0.0114 - val_accuracy: 0.0192
Epoch 384/500
312/312 [==============================] - 1s 3ms/step - loss: 0.0472 - accuracy: 0.0054 - val_loss: 0.0115 - val_accuracy: 0.0187
Epoch 385/500
312/312 [==============================] - 1s 3ms/step - loss: 0.0471 - accuracy: 0.0034 - val_loss: 0.0114 - val_accuracy: 0.1075
Epoch 386/500
312/312 [==============================] - 1s 3ms/step - loss: 0.0471 - accuracy: 0.0150 - val_loss: 0.0115 - val_accuracy: 0.0187
Epoch 387/500
312/312 [==============================] - 1s 3ms/step - loss: 0.0472 - accuracy: 0.0102 - val_loss: 0.0115 - val_accuracy: 0.0084
Epoch 388/500
312/312 [==============================] - 1s 3ms/step - loss: 0.0471 - accuracy: 0.0058 - val_loss: 0.0114 - val_accuracy: 0.0519
Epoch 389/500
312/312 [==============================] - 1s 3ms/step - loss: 0.0471 - accuracy: 0.0024 - val_loss: 0.0114 - val_ac

312/312 [==============================] - 1s 3ms/step - loss: 0.0470 - accuracy: 0.0060 - val_loss: 0.0115 - val_accuracy: 0.0944
Epoch 440/500
312/312 [==============================] - 1s 3ms/step - loss: 0.0470 - accuracy: 0.0126 - val_loss: 0.0115 - val_accuracy: 0.0393
Epoch 441/500
312/312 [==============================] - 1s 3ms/step - loss: 0.0470 - accuracy: 0.0102 - val_loss: 0.0115 - val_accuracy: 0.0065
Epoch 442/500
312/312 [==============================] - 1s 3ms/step - loss: 0.0472 - accuracy: 0.0026 - val_loss: 0.0115 - val_accuracy: 0.0248
Epoch 443/500
312/312 [==============================] - 1s 3ms/step - loss: 0.0470 - accuracy: 0.0058 - val_loss: 0.0115 - val_accuracy: 0.0187
Epoch 444/500
312/312 [==============================] - 1s 3ms/step - loss: 0.0470 - accuracy: 0.0058 - val_loss: 0.0115 - val_accuracy: 0.0103
Epoch 445/500
312/312 [==============================] - 1s 3ms/step - loss: 0.0471 - accuracy: 0.0016 - val_loss: 0.0115 - val_accuracy: 0.0552

312/312 [==============================] - 1s 3ms/step - loss: 0.0470 - accuracy: 0.0034 - val_loss: 0.0116 - val_accuracy: 0.0243
Epoch 496/500
312/312 [==============================] - 1s 3ms/step - loss: 0.0469 - accuracy: 0.0022 - val_loss: 0.0116 - val_accuracy: 0.0140
Epoch 497/500
312/312 [==============================] - 1s 3ms/step - loss: 0.0470 - accuracy: 0.0064 - val_loss: 0.0116 - val_accuracy: 0.0271
Epoch 498/500
312/312 [==============================] - 1s 4ms/step - loss: 0.0473 - accuracy: 0.0022 - val_loss: 0.0116 - val_accuracy: 0.0388
Epoch 499/500
312/312 [==============================] - 1s 3ms/step - loss: 0.0471 - accuracy: 0.0040 - val_loss: 0.0116 - val_accuracy: 0.0065
Epoch 500/500
312/312 [==============================] - 1s 3ms/step - loss: 0.0470 - accuracy: 0.0024 - val_loss: 0.0115 - val_accuracy: 0.0379


In [73]:
model.fit(Xscaled, y,
                    validation_split=0.3,
                    epochs=500,
                    batch_size=16, 
                    verbose=1)

Epoch 1/500
312/312 [==============================] - 1s 4ms/step - loss: 0.0470 - accuracy: 0.0032 - val_loss: 0.0115 - val_accuracy: 0.0014
Epoch 2/500
312/312 [==============================] - 1s 3ms/step - loss: 0.0470 - accuracy: 0.0014 - val_loss: 0.0116 - val_accuracy: 0.0089
Epoch 3/500
312/312 [==============================] - 1s 3ms/step - loss: 0.0470 - accuracy: 0.0014 - val_loss: 0.0115 - val_accuracy: 0.0094
Epoch 4/500
312/312 [==============================] - 1s 4ms/step - loss: 0.0469 - accuracy: 0.0014 - val_loss: 0.0115 - val_accuracy: 0.0089
Epoch 5/500
312/312 [==============================] - 1s 3ms/step - loss: 0.0470 - accuracy: 0.0032 - val_loss: 0.0116 - val_accuracy: 0.0094
Epoch 6/500
312/312 [==============================] - 1s 3ms/step - loss: 0.0469 - accuracy: 0.0018 - val_loss: 0.0116 - val_accuracy: 0.0098
Epoch 7/500
312/312 [==============================] - 1s 3ms/step - loss: 0.0469 - accuracy: 0.0024 - val_loss: 0.0116 - val_accuracy: 0.0084

Epoch 58/500
312/312 [==============================] - 1s 4ms/step - loss: 0.0468 - accuracy: 0.0100 - val_loss: 0.0116 - val_accuracy: 0.0248
Epoch 59/500
312/312 [==============================] - 1s 3ms/step - loss: 0.0470 - accuracy: 0.0036 - val_loss: 0.0116 - val_accuracy: 0.0425
Epoch 60/500
312/312 [==============================] - 1s 3ms/step - loss: 0.0469 - accuracy: 0.0032 - val_loss: 0.0116 - val_accuracy: 0.0608
Epoch 61/500
312/312 [==============================] - 1s 3ms/step - loss: 0.0469 - accuracy: 0.0048 - val_loss: 0.0116 - val_accuracy: 0.0290
Epoch 62/500
312/312 [==============================] - 1s 3ms/step - loss: 0.0469 - accuracy: 0.0064 - val_loss: 0.0116 - val_accuracy: 0.0051
Epoch 63/500
312/312 [==============================] - 1s 3ms/step - loss: 0.0469 - accuracy: 0.0036 - val_loss: 0.0116 - val_accuracy: 0.0393
Epoch 64/500
312/312 [==============================] - 1s 3ms/step - loss: 0.0470 - accuracy: 0.0052 - val_loss: 0.0116 - val_accuracy:

Epoch 115/500
312/312 [==============================] - 1s 4ms/step - loss: 0.0468 - accuracy: 0.0028 - val_loss: 0.0116 - val_accuracy: 0.0519
Epoch 116/500
312/312 [==============================] - 1s 3ms/step - loss: 0.0468 - accuracy: 0.0036 - val_loss: 0.0117 - val_accuracy: 0.0224
Epoch 117/500
312/312 [==============================] - 1s 3ms/step - loss: 0.0468 - accuracy: 0.0030 - val_loss: 0.0116 - val_accuracy: 0.0355
Epoch 118/500
312/312 [==============================] - 1s 3ms/step - loss: 0.0468 - accuracy: 0.0046 - val_loss: 0.0116 - val_accuracy: 0.0215
Epoch 119/500
312/312 [==============================] - 1s 4ms/step - loss: 0.0468 - accuracy: 0.0044 - val_loss: 0.0117 - val_accuracy: 0.0159
Epoch 120/500
312/312 [==============================] - 1s 4ms/step - loss: 0.0469 - accuracy: 0.0024 - val_loss: 0.0118 - val_accuracy: 0.0355
Epoch 121/500
312/312 [==============================] - 1s 4ms/step - loss: 0.0469 - accuracy: 0.0054 - val_loss: 0.0116 - val_ac

312/312 [==============================] - 1s 3ms/step - loss: 0.0469 - accuracy: 0.0020 - val_loss: 0.0116 - val_accuracy: 0.0079
Epoch 172/500
312/312 [==============================] - 1s 3ms/step - loss: 0.0471 - accuracy: 0.0024 - val_loss: 0.0117 - val_accuracy: 0.0145
Epoch 173/500
312/312 [==============================] - 1s 3ms/step - loss: 0.0467 - accuracy: 0.0020 - val_loss: 0.0118 - val_accuracy: 0.0192
Epoch 174/500
312/312 [==============================] - 1s 3ms/step - loss: 0.0468 - accuracy: 0.0022 - val_loss: 0.0117 - val_accuracy: 0.0276
Epoch 175/500
312/312 [==============================] - 1s 3ms/step - loss: 0.0468 - accuracy: 0.0032 - val_loss: 0.0117 - val_accuracy: 0.0332
Epoch 176/500
312/312 [==============================] - 1s 3ms/step - loss: 0.0468 - accuracy: 0.0024 - val_loss: 0.0117 - val_accuracy: 0.0468
Epoch 177/500
312/312 [==============================] - 1s 3ms/step - loss: 0.0469 - accuracy: 0.0040 - val_loss: 0.0116 - val_accuracy: 0.0397

312/312 [==============================] - 1s 4ms/step - loss: 0.0469 - accuracy: 0.0028 - val_loss: 0.0117 - val_accuracy: 0.0206
Epoch 228/500
312/312 [==============================] - 1s 3ms/step - loss: 0.0468 - accuracy: 0.0032 - val_loss: 0.0117 - val_accuracy: 0.0136
Epoch 229/500
312/312 [==============================] - 1s 4ms/step - loss: 0.0469 - accuracy: 0.0026 - val_loss: 0.0117 - val_accuracy: 0.0215
Epoch 230/500
312/312 [==============================] - 1s 5ms/step - loss: 0.0468 - accuracy: 0.0040 - val_loss: 0.0117 - val_accuracy: 0.0117
Epoch 231/500
312/312 [==============================] - 1s 4ms/step - loss: 0.0468 - accuracy: 0.0026 - val_loss: 0.0117 - val_accuracy: 0.0061
Epoch 232/500
312/312 [==============================] - 2s 5ms/step - loss: 0.0468 - accuracy: 0.0056 - val_loss: 0.0117 - val_accuracy: 0.0122
Epoch 233/500
312/312 [==============================] - 1s 4ms/step - loss: 0.0467 - accuracy: 0.0016 - val_loss: 0.0118 - val_accuracy: 0.0108

312/312 [==============================] - 1s 4ms/step - loss: 0.0467 - accuracy: 0.0016 - val_loss: 0.0118 - val_accuracy: 0.0187
Epoch 284/500
312/312 [==============================] - 1s 4ms/step - loss: 0.0467 - accuracy: 0.0030 - val_loss: 0.0118 - val_accuracy: 0.0070
Epoch 285/500
312/312 [==============================] - 1s 4ms/step - loss: 0.0467 - accuracy: 0.0020 - val_loss: 0.0118 - val_accuracy: 0.0150
Epoch 286/500
312/312 [==============================] - 1s 4ms/step - loss: 0.0467 - accuracy: 0.0028 - val_loss: 0.0118 - val_accuracy: 0.0173
Epoch 287/500
312/312 [==============================] - 1s 4ms/step - loss: 0.0468 - accuracy: 0.0016 - val_loss: 0.0118 - val_accuracy: 0.0145
Epoch 288/500
312/312 [==============================] - 1s 4ms/step - loss: 0.0467 - accuracy: 0.0030 - val_loss: 0.0117 - val_accuracy: 0.0131
Epoch 289/500
312/312 [==============================] - 1s 4ms/step - loss: 0.0469 - accuracy: 0.0018 - val_loss: 0.0117 - val_accuracy: 0.0131

312/312 [==============================] - 1s 3ms/step - loss: 0.0467 - accuracy: 0.0016 - val_loss: 0.0118 - val_accuracy: 0.0154
Epoch 340/500
312/312 [==============================] - 1s 3ms/step - loss: 0.0468 - accuracy: 0.0012 - val_loss: 0.0117 - val_accuracy: 0.0154
Epoch 341/500
312/312 [==============================] - 2s 5ms/step - loss: 0.0468 - accuracy: 0.0028 - val_loss: 0.0118 - val_accuracy: 0.0103
Epoch 342/500
312/312 [==============================] - 2s 6ms/step - loss: 0.0469 - accuracy: 0.0018 - val_loss: 0.0118 - val_accuracy: 0.0079
Epoch 343/500
312/312 [==============================] - 2s 5ms/step - loss: 0.0467 - accuracy: 0.0010 - val_loss: 0.0116 - val_accuracy: 0.0108
Epoch 344/500
312/312 [==============================] - 1s 5ms/step - loss: 0.0467 - accuracy: 0.0014 - val_loss: 0.0118 - val_accuracy: 0.0047
Epoch 345/500
312/312 [==============================] - 1s 3ms/step - loss: 0.0468 - accuracy: 6.0108e-04 - val_loss: 0.0117 - val_accuracy: 0.

312/312 [==============================] - 1s 5ms/step - loss: 0.0467 - accuracy: 0.0036 - val_loss: 0.0118 - val_accuracy: 0.0201
Epoch 396/500
312/312 [==============================] - 1s 5ms/step - loss: 0.0467 - accuracy: 0.0032 - val_loss: 0.0117 - val_accuracy: 0.0168
Epoch 397/500
312/312 [==============================] - 1s 4ms/step - loss: 0.0467 - accuracy: 0.0020 - val_loss: 0.0117 - val_accuracy: 0.0248
Epoch 398/500
312/312 [==============================] - 1s 4ms/step - loss: 0.0466 - accuracy: 0.0028 - val_loss: 0.0118 - val_accuracy: 0.0262
Epoch 399/500
312/312 [==============================] - 1s 3ms/step - loss: 0.0467 - accuracy: 0.0028 - val_loss: 0.0118 - val_accuracy: 0.0154
Epoch 400/500
312/312 [==============================] - 1s 3ms/step - loss: 0.0466 - accuracy: 0.0026 - val_loss: 0.0117 - val_accuracy: 0.0243
Epoch 401/500
312/312 [==============================] - 1s 3ms/step - loss: 0.0467 - accuracy: 0.0036 - val_loss: 0.0118 - val_accuracy: 0.0150

312/312 [==============================] - 1s 3ms/step - loss: 0.0468 - accuracy: 0.0012 - val_loss: 0.0118 - val_accuracy: 0.0159
Epoch 452/500
312/312 [==============================] - 1s 4ms/step - loss: 0.0467 - accuracy: 0.0024 - val_loss: 0.0118 - val_accuracy: 0.0154
Epoch 453/500
312/312 [==============================] - 1s 4ms/step - loss: 0.0467 - accuracy: 0.0018 - val_loss: 0.0118 - val_accuracy: 0.0304
Epoch 454/500
312/312 [==============================] - 1s 4ms/step - loss: 0.0466 - accuracy: 0.0054 - val_loss: 0.0119 - val_accuracy: 0.0089
Epoch 455/500
312/312 [==============================] - 1s 4ms/step - loss: 0.0467 - accuracy: 0.0034 - val_loss: 0.0118 - val_accuracy: 0.0323
Epoch 456/500
312/312 [==============================] - 1s 4ms/step - loss: 0.0466 - accuracy: 0.0024 - val_loss: 0.0118 - val_accuracy: 0.0248
Epoch 457/500
312/312 [==============================] - 1s 3ms/step - loss: 0.0468 - accuracy: 0.0018 - val_loss: 0.0119 - val_accuracy: 0.0430

In [76]:
spike_flags[spike_flags == 1].count()

10.0     6959
11.0     6926
12.0     6930
13.0     6933
14.0     6929
         ... 
97.0     7107
98.0     7104
99.0     7108
100.0    7101
101.0    7113
Length: 92, dtype: int64